## 特征提取

In [28]:
import glob
import pandas as pd
import pickle

PATH = '/home/wx/work/VideoCopyDetection/'
TRAIN_PATH = PATH + 'train/'
TEST_PATH = PATH + 'test/'
QUERY_PATH = TRAIN_PATH + 'query/'
REFER_PATH = TRAIN_PATH + 'refer/'
QUERY_FRAME_PATH = TRAIN_PATH + 'query_frame/'
REFER_FRAME_PATH = TRAIN_PATH + 'refer_frame/'
TEST_QUERY_PATH = TEST_PATH + 'query/'
TEST_QUERY_FRAME_PATH = TEST_PATH + 'query_frame/'
CODE_DIR = PATH + 'code/'

In [29]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.dataset import Dataset

import logging
logging.basicConfig(level = logging.DEBUG, filename = 'example.log',
                    format = '%(asctime)s - %(filename)s[line:%(lineno)d]: %(message)s')  # 

class QRDataset(Dataset):
    def __init__(self, img_path, transform = None):
        self.img_path = img_path
        print(self.img_path)
        self.img_label = np.zeros(len(img_path))
    
        if transform is not None:
            self.transform = transform
        else:
            self.transform = None
    
    def __getitem__(self, index):
        img = Image.open(self.img_path[index])
        
        if self.transform is not None:
            img = self.transform(img)
        
        return img, self.img_path[index]

    def __len__(self):
        return len(self.img_path)
    
model = models.resnet18(pretrained = True).cuda()

def extract_feature(path):
    if not isinstance(path, list):
        path = [path]
    
    data_loader = torch.utils.data.DataLoader(
        QRDataset(path, 
                transforms.Compose([
                            transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        ), batch_size = 40, shuffle = False, num_workers = 10,
    )
    
    img_feature = []
    with torch.no_grad():
        for batch_data in tqdm_notebook(data_loader):
            batch_x, batch_y = batch_data
            
            # print(batch_y[:10])
            batch_x = Variable(batch_x).cuda()
            feature_pred = model(batch_x)

            # max-pooling
            # feature_pred = F.max_pool2d(feature_pred, kernel_size=(24, 32))
            
            # ave-pooling
            # feature_pred = F.avg_pool2d(feature_pred, kernel_size=(24, 32))[:, :, 0, 0]
            
            #print(feature_pred.shape, batch_x.shape)
            feature_pred = feature_pred.data.cpu().numpy()
            # feature_pred = feature_pred.max(-1).max(-1)
            
            # feature_pred = feature_pred.reshape((-1, 512))
            img_feature.append(feature_pred)
            
            del feature_pred
            # img_feature.append(feature_pred)
            
    img_feature = np.vstack(img_feature)
    return img_feature

In [34]:
# 抽取query关键帧特征
test_query_features = extract_feature(list(query_imgs_path[:]))

b98a-11e9-b8ed-fa163e063a2b_00000077.200000014.600000.jpg', '/home/wx/work/VideoCopyDetection/test/query_frame/ff586dc4-b98a-11e9-b8ed-fa163e063a2b/ff586dc4-b98a-11e9-b8ed-fa163e063a2b_00000078.800000024.866667.jpg', '/home/wx/work/VideoCopyDetection/test/query_frame/ff586dc4-b98a-11e9-b8ed-fa163e063a2b/ff586dc4-b98a-11e9-b8ed-fa163e063a2b_00000080.200000057.266667.jpg', '/home/wx/work/VideoCopyDetection/test/query_frame/ff586dc4-b98a-11e9-b8ed-fa163e063a2b/ff586dc4-b98a-11e9-b8ed-fa163e063a2b_00000081.300000024.333333.jpg', '/home/wx/work/VideoCopyDetection/test/query_frame/ff586dc4-b98a-11e9-b8ed-fa163e063a2b/ff586dc4-b98a-11e9-b8ed-fa163e063a2b_00000082.400000149.600000.jpg', '/home/wx/work/VideoCopyDetection/test/query_frame/ff586dc4-b98a-11e9-b8ed-fa163e063a2b/ff586dc4-b98a-11e9-b8ed-fa163e063a2b_00000084.500000012.866667.jpg', '/home/wx/work/VideoCopyDetection/test/query_frame/ff586dc4-b98a-11e9-b8ed-fa163e063a2b/ff586dc4-b98a-11e9-b8ed-fa163e063a2b_00000087.800000081.333333.jpg'

In [12]:
# 抽取refer关键帧特征
refer_features = extract_feature(list(refer_imgs_path[:]))

0000913.900000461.200000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000923.700000257.760000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000933.700000098.280000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000934.700000514.920000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000942.000000403.200000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000942.900000232.640000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000943.000000058.560000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000947.000000325.240000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000951.700000959.320000.jpg', '/home/wx/work/VideoCopyDetection/train/refer_frame/666051400/666051400_00000959.300000788.160000.jpg', '/home/wx/work/VideoCopyDetection

In [30]:
import os, sys, codecs
import glob

import numpy as np
import cv2

from sklearn.preprocessing import normalize as sknormalize
from sklearn.decomposition import PCA

def normalize(x, copy=False):
    """
    A helper function that wraps the function of the same name in sklearn.
    This helper handles the case of a single column vector.
    """
    if type(x) == np.ndarray and len(x.shape) == 1:
        return np.squeeze(sknormalize(x.reshape(1,-1), copy = copy))
        #return np.squeeze(x / np.sqrt((x ** 2).sum(-1))[..., np.newaxis])
    else:
        return sknormalize(x, copy = copy)
        #return x / np.sqrt((x ** 2).sum(-1))[..., np.newaxis]

In [31]:
# 特征归一化

test_query_features = normalize(test_query_features)
refer_features = normalize(refer_features)

In [32]:
# 保存query关键帧特征

with open(PATH + 'var/test_query_features.txt', 'wb') as f:
    pickle.dump(test_query_features, f)

In [33]:
# 保存refer关键帧特征

with open(PATH + 'var/refer_features.txt', 'wb') as f:
    pickle.dump(refer_features, f)